In [1]:
from meb import utils
from meb import datasets
from meb import models
from meb import core

import os
from tqdm import tqdm
from typing import Callable, List, Tuple, Union
from functools import partial

import numpy as np
from numba import jit
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torchvision import transforms
import timm



pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

In [4]:
c = datasets.CrossDataset(resize=112, color=True, preload=True)
df = c.data_frame
data = c.data

In [ ]:
#interpolate samples with less than 8 frames
n_frames = 8
for i, video in enumerate(data):
    if video.shape[0] < n_frames:
        new_shape = (n_frames,) + video.shape[1:-1]
        video = torch.tensor(video).permute(3, 0, 1, 2).unsqueeze(0).float()
        new_video = F.interpolate(video, size=new_shape, mode="trilinear")
        data[i] = new_video.squeeze(0).permute(1, 2, 3, 0).byte().numpy()

In [6]:
# Create a function that returns the model as it needs to be modified
def r2plus1d(num_classes: int):
    model = torchvision.models.video.r2plus1d_18(weights=torchvision.models.video.R2Plus1D_18_Weights.DEFAULT)
    model.fc = nn.Linear(in_features=512, out_features=num_classes)
    return model

In [8]:
class Config(core.Config):
    device = torch.device("cuda:1")
    epochs = 100
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary"),
    ]
    batch_size = 32
    train_transform = {
        "spatial": None,
        "temporal": datasets.UniformTemporalSubsample(8),
    }
    test_transform = {
        "spatial": None,
        "temporal": datasets.UniformTemporalSubsample(8),
    }
    model = partial(r2plus1d, num_classes=len(core.Config.action_units))

In [10]:
core.CrossDatasetValidator(Config).validate_n_times(df, data, n_times=5)

  0%|                                                     | 0/5 [00:00<?, ?it/s]/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
 20%|███████▏                            | 1/5 [16:46:19<67:05:19, 60379.80s/it]/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
74.0 & 71.5 & 80.1 & 60.7 & 51.4 & 56.6 & 54.1 & 55.1 & 59.2 & 58.8 & 54.1 & 70.0 & 62.1

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
62.9 & 62.8 & 64.2 & 57.3 & 64.6 & 58.1 & 61.7
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
54.2 & 49.4 & 72.7 & 26.9 & 5.2 & 20.4 & 11.7 & 12.4 & 23.3 & 25.5 & 9.7 & 41.9 & 29.5

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
27.6 & 30.0 & 33.0 & 21.8 & 34.4 & 21.4 & 28.0


In [10]:
out = core.CrossDatasetValidation(Config).validate(df, data)

100%|██████████████████████████████████████| 100/100 [5:10:13<00:00, 186.14s/it]
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9919 | test_mean: 0.7202
Test per AU: [('AU1', 84.95), ('AU2', 82.93), ('AU4', 81.8), ('AU5', 100.0), ('AU6', 57.86), ('AU7', 49.67), ('AU9', 68.18), ('AU10', 48.92), ('AU12', 68.32), ('AU14', 71.54), ('AU15', 73.67), ('AU17', 76.44)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9848 | test_mean: 0.409
Test per AU: [('AU1', 73.17), ('AU2', 68.75), ('AU4', 78.15), ('AU5', 0.0), ('AU6', 18.18), ('AU7', 10.0), ('AU9', 45.28), ('AU10', 0.0), ('AU12', 40.0), ('AU14', 48.65), ('AU15', 52.63), ('AU17', 56.0)]



100%|██████████████████████████████████████| 100/100 [5:58:14<00:00, 214.94s/it]


MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 1.0 | test_mean: 0.699
Test per AU: [('AU1', 96.84), ('AU2', 81.84), ('AU4', 86.62), ('AU5', 49.71), ('AU6', 55.94), ('AU7', 70.83), ('AU9', 53.17), ('AU10', 48.39), ('AU12', 67.38), ('AU14', 68.95), ('AU15', 78.14), ('AU17', 81.02)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 1.0 | test_mean: 0.4395
Test per AU: [('AU1', 94.34), ('AU2', 66.67), ('AU4', 85.47), ('AU5', 0.0), ('AU6', 14.29), ('AU7', 48.28), ('AU9', 10.0), ('AU10', 0.0), ('AU12', 41.51), ('AU14', 43.48), ('AU15', 58.33), ('AU17', 65.0)]



100%|██████████████████████████████████████| 100/100 [3:14:24<00:00, 116.64s/it]


...

MultiLabelF1Score
Dataset: fourd, n=267 | train_mean: 1.0 | test_mean: 0.6111
Test per AU: [('AU1', 80.89), ('AU2', 80.95), ('AU4', 80.99), ('AU5', 59.24), ('AU6', 51.48), ('AU7', 40.0), ('AU9', 49.24), ('AU10', 49.81), ('AU12', 54.94), ('AU14', 53.61), ('AU15', 49.43), ('AU17', 82.75)]

MultiLabelF1Score
Dataset: fourd, n=267 | train_mean: 1.0 | test_mean: 0.2798
Test per AU: [('AU1', 67.5), ('AU2', 68.75), ('AU4', 74.58), ('AU5', 20.0), ('AU6', 7.69), ('AU7', 0.0), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 20.0), ('AU14', 10.53), ('AU15', 0.0), ('AU17', 66.67)]



100%|██████████████████████████████████████| 100/100 [2:59:49<00:00, 107.89s/it]


MultiLabelF1Score
Dataset: mmew, n=300 | train_mean: 1.0 | test_mean: 0.6685
Test per AU: [('AU1', 86.93), ('AU2', 88.01), ('AU4', 88.86), ('AU5', 58.75), ('AU6', 49.15), ('AU7', 57.57), ('AU9', 49.32), ('AU10', 67.44), ('AU12', 66.78), ('AU14', 62.98), ('AU15', 49.32), ('AU17', 77.09)]

MultiLabelF1Score
Dataset: mmew, n=300 | train_mean: 1.0 | test_mean: 0.3833
Test per AU: [('AU1', 77.78), ('AU2', 79.55), ('AU4', 84.82), ('AU5', 28.92), ('AU6', 0.0), ('AU7', 22.64), ('AU9', 0.0), ('AU10', 40.74), ('AU12', 39.22), ('AU14', 30.77), ('AU15', 0.0), ('AU17', 55.56)]



100%|███████████████████████████████████████| 100/100 [2:01:56<00:00, 73.17s/it]


MultiLabelF1Score
Dataset: casme3a, n=860 | train_mean: 0.9997 | test_mean: 0.6165
Test per AU: [('AU1', 75.06), ('AU2', 76.62), ('AU4', 77.17), ('AU5', 56.56), ('AU6', 49.14), ('AU7', 49.77), ('AU9', 50.23), ('AU10', 61.79), ('AU12', 54.99), ('AU14', 57.86), ('AU15', 56.29), ('AU17', 74.34)]

MultiLabelF1Score
Dataset: casme3a, n=860 | train_mean: 0.9995 | test_mean: 0.29
Test per AU: [('AU1', 56.77), ('AU2', 58.71), ('AU4', 67.47), ('AU5', 21.56), ('AU6', 0.0), ('AU7', 12.56), ('AU9', 3.64), ('AU10', 25.0), ('AU12', 11.76), ('AU14', 27.23), ('AU15', 13.33), ('AU17', 50.0)]

Final results

MultiLabelF1Score
All AUs:  [('AU1', 81.02), ('AU2', 80.56), ('AU4', 81.76), ('AU5', 59.24), ('AU6', 53.39), ('AU7', 54.03), ('AU9', 57.85), ('AU10', 63.08), ('AU12', 63.93), ('AU14', 61.27), ('AU15', 65.8), ('AU17', 77.73)]
Mean:  66.64


MultiLabelF1Score
All AUs:  [('AU1', 66.8), ('AU2', 65.86), ('AU4', 75.1), ('AU5', 24.0), ('AU6', 9.26), ('AU7', 20.04), ('AU9', 19.16), ('AU10', 28.33), ('AU12',